Nobody, but the NUMBER 1 RAPPER IN THE WORLD
Gonna load to entire load at once and see how it goes.
Nvm there are a lot.
Ill load all of 1 first.

In [9]:
# Open and read the contents of the file


In [18]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import glob
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np


import nltk

# nltk.download('punkt_tab')


programs_dict = {}
limit = 20
for i in range(104):
    folder_path = 'Thesis Data/ProgramData/' + str(i)
    files = glob.glob(os.path.join(folder_path, '*'))
    count = 0
    for file in files:
        with open(file, 'r') as f:
            if i not in programs_dict.keys():
                programs_dict[i] = [f.read()]
            else:
                programs_dict.get(i).append(f.read())
        count += 1
        if count >+ limit: break


class TextDataset(Dataset):
    def __init__(self, programs_dict, tokenizer):
        self.texts = []
        self.labels = []
        self.max_len = 0
        self.tokenizer = tokenizer
        for key in programs_dict.keys():
            for ls in programs_dict[key]:
                if len(ls) > self.max_len: self.max_len = len(ls)
            self.texts += programs_dict[key]
            self.labels += [key] * len(programs_dict[key])

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize and pad sequences
        sequence = self.tokenizer.texts_to_sequences([self.texts[idx]])[0]
        sequence = pad_sequences([sequence], maxlen=self.max_len, padding='post')[0]
        label = self.labels[idx]  # Label as an integer
        return torch.tensor(sequence, dtype=torch.long), torch.tensor(label, dtype=torch.long)

# Tokenize the corpus
tokenized_corpus_list = [[word_tokenize(item.lower()) for item in ls] for ls in programs_dict.values()]
tokenized_corpus = []
for inner in tokenized_corpus_list : tokenized_corpus += inner
# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_corpus, vector_size=100, window=5, min_count=1, workers=4)
# Save the model
word2vec_model.save("word2vec.model")

tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_corpus)
sequences = tokenizer.texts_to_sequences(tokenized_corpus)
word_index = tokenizer.word_index
# Loading into dataloader class using the same tokenizer
train_dataset = TextDataset(programs_dict=programs_dict, tokenizer=tokenizer)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)

embedding_dim = 100
word_vectors = word2vec_model.wv
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    if word in word_vectors:
        embedding_matrix[i] = word_vectors[word]
    else:
        # Randomly initialize the embedding for out-of-vocabulary words
        embedding_matrix[i] = np.random.normal(size=(embedding_dim,))




; 1
) 2
( 3
[ 4
] 5
, 6
? 7
i 8
< 9
} 10
{ 11
* 12
& 13
int 14
% 15
a 16
if 17
`` 18
for 19
j 20
> 21
'' 22
n 23
' 24
0 25
= 26
d 27
i++ 28
b 29
printf 30
k 31
i=0 32
c 33
else 34
scanf 35
s 36
return 37
m 38
main 39
! 40
1 41
j++ 42
cout 43
// 44
char 45
== 46
break 47
p 48
=0 49
100 50
j=0 51
x 52
d\n 53
cin 54
endl 55
2 56
: 57
t 58
num 59
+ 60
-- 61
5 62
- 63
l 64
y 65
while 66
i=1 67
str 68
void 69
sz 70
=a 71
3 72
e 73
struct 74
10 75
f 76
double 77
|| 78
k++ 79
z 80
/ 81
i+1 82
q 83
case 84
sum 85
max 86
=n 87
j+1 88
k=0 89
++ 90
n-1 91
='\0 92
4 93
==0 94
1000 95
7 96
i-1 97
temp 98
p1- 99
9 100
=1 101
gets 102
len 103
row 104
next 105
g 106
sizeof 107
\n 108
r 109
continue 110
j=1 111
strlen 112
p- 113
h 114
word 115
==1 116
col 117
stu 118
getchar 119
=b 120
j-1 121
w 122
malloc 123
no 124
min 125
+= 126
=c 127
count 128
year 129
6 130
lf 131
20 132
m=0 133
50 134
300 135
101 136
day 137
flag 138
s1 139
s\n 140
-1 141
age 142
student 143
zfc 144
n=0 145
+j 146
200 147
p+i 148

In [11]:
# iwanne be
# PERFECT
# I KNOW ILL neva be
# perfect

# IF YOU COULD CHOOSE ONLY 1 THING TO FIX
# worthless...


class LSTMModel(nn.Module):
    def __init__(self, embedding_matrix):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_matrix), freeze=True)
        # freeze=True makes it so that the embeddings are NOT updated during training
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=128, batch_first=True)
        self.fc = nn.Linear(128, 104)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.embedding(x)
        x, (h_n, c_n) = self.lstm(x)
        x = self.fc(h_n[-1])
        return x

# Example instantiation
model = LSTMModel(embedding_matrix)

criterion = nn.CrossEntropyLoss()  # For multi class classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Clear gradients
        for name, param in model.named_parameters():
            if param.requires_grad:
                print(name, param.grad.norm())
        # Forward pass
        outputs = model(inputs)  # Model prediction
        loss = criterion(outputs, labels)  # Compute loss

        # Backpropagation
        loss.backward()  # Compute gradients
        optimizer.step()  # Update model parameters
        print(f'Batch Loss: {loss.item()}')
        running_loss += loss.item() * inputs.size(0)  # Track loss

    epoch_loss = running_loss / len(train_loader.dataset)  # Average loss per epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')







Batch Loss: 4.649692058563232
Batch Loss: 4.656373500823975


KeyboardInterrupt: 